In [2]:
import pandas as pd
import csv
import json

In [3]:
with open('C:/Users/weijin.ang/Code/transactions_30042023.json', 'r') as f:
  data = json.load(f)
#data

In [4]:
# normalize events
df = pd.json_normalize(data, 'Result')

# explode all columns with lists of dicts
df = df.apply(lambda x: x.explode()).reset_index(drop=True)  # df.apply(pd.Series.explode).reset_index(drop=True) also works

# list of columns with dicts
cols_to_normalize = ['rental']

# if there are keys, which will become column names, overlap with excising column names
# add the current column name as a prefix
normalized = list()
for col in cols_to_normalize:
    
    d = pd.json_normalize(df[col], sep='_')
    d.columns = [f'{col}_{v}' for v in d.columns]
    normalized.append(d.copy())

# combine df with the normalized columns , concat both lists
df = pd.concat([df] + normalized, axis=1).drop(columns=cols_to_normalize)

In [4]:
df
#df.to_csv('transaction_v2.csv', index=False)

,street,x,project,y,rental_areaSqm,rental_leaseDate,rental_propertyType,rental_district,rental_areaSqft,rental_noOfBedRoom,rental_rent
0,ELLIOT ROAD,38865.17182,ELLIOT AT THE EAST COAST,32621.92134,210-220,0123,Non-landed Properties,15,2300-2400,3,6000
1,ELLIOT ROAD,38865.17182,ELLIOT AT THE EAST COAST,32621.92134,120-130,0123,Non-landed Properties,15,1300-1400,3,4600
2,ELLIOT ROAD,38865.17182,ELLIOT AT THE EAST COAST,32621.92134,120-130,0123,Non-landed Properties,15,1300-1400,3,3700
3,ELLIOT ROAD,38865.17182,ELLIOT AT THE EAST COAST,32621.92134,130-140,0223,Non-landed Properties,15,1400-1500,3,4800
4,ELLIOT ROAD,38865.17182,ELLIOT AT THE EAST COAST,32621.92134,130-140,0223,Non-landed Properties,15,1400-1500,3,6200
...,...,...,...,...,...,...,...,...,...,...,...
19785,PERUMAL ROAD,30477.87296,UPTOWN @ FARRER,32796.58779,40-50,0223,Non-landed Properties,08,500-600,2,4000
19786,PERUMAL ROAD,30477.87296,UPTOWN @ FARRER,32796.58779,40-50,0323,Non-landed Properties,08,500-600,2,4500
19787,PERUMAL ROAD,30477.87296,UPTOWN @ FARRER,32796.58779,40-50,0323,Non-landed Properties,08,500-600,2,4200
19788,PERUMAL ROAD,30477.87296,UPTOWN @ FARRER,32796.58779,100-110,0323,Non-landed Properties,08,1100-1200,3,6100


In [5]:
df = df[df.rental_propertyType.isin(["Non-landed Properties", "Executive Condominium"])]

In [6]:
df = df[df.rental_noOfBedRoom.isin(["1","2","3","4","5"])]

In [7]:
#df_50th = df.groupby(['rental_district']).mean()
df_25th = df.groupby(['rental_district', 'rental_noOfBedRoom']).quantile(.25).rename(columns={"rental_rent": "rental_rent_25th"})
df_50th = df.groupby(['rental_district', 'rental_noOfBedRoom']).quantile(.50).rename(columns={"rental_rent": "rental_rent_50th"})
df_75th = df.groupby(['rental_district', 'rental_noOfBedRoom']).quantile(.75).rename(columns={"rental_rent": "rental_rent_75th"})

C:\Users\weijin.ang\AppData\Local\Temp\ipykernel_22044\1419871308.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.quantile is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_25th = df.groupby(['rental_district', 'rental_noOfBedRoom']).quantile(.25).rename(columns={"rental_rent": "rental_rent_25th"})
C:\Users\weijin.ang\AppData\Local\Temp\ipykernel_22044\1419871308.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.quantile is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_50th = df.groupby(['rental_district', 'rental_noOfBedRoom']).quantile(.50).rename(columns={"rental_rent": "rental_rent_50th"})
C:\Users\weijin.ang\AppData\Local\Temp\ipykernel_22044\1419871308.py:4: FutureWarning: The default value of nu

In [8]:
df = pd.concat([df_25th] + [df_50th] + [df_75th], axis=1)
df

rental_rent_25th  rental_rent_50th  \
rental_district rental_noOfBedRoom                                       
01              1                             4300.0            4900.0   
                2                             5950.0            6850.0   
                3                             6775.0            9400.0   
                4                            13000.0           16750.0   
                5                            50000.0           50000.0   
...                                              ...               ...   
27              4                             4100.0            4800.0   
28              1                             2500.0            3000.0   
                2                             3200.0            3500.0   
                3                             3700.0            4000.0   
                4                             4800.0            5000.0   

                                    rental_rent_75th  
rental_district rental_noOfBedRoom                    
01              1                             5500.0  
                2                             8000.0  
                3                            11000.0  
                4                            18500.0  
                5                            50000.0  
...                                              ...  
27              4                             5300.0  
28              1                             3025.0  
                2                             3700.0  
                3                             4500.0  
                4                             5288.0  

[124 rows x 3 columns]

In [9]:
df = pd.pivot_table(df, values = ['rental_rent_25th','rental_rent_25th','rental_rent_75th'], index=['rental_district'], columns = 'rental_noOfBedRoom').reset_index()
df


rental_district rental_rent_25th                           \
rental_noOfBedRoom                                1       2       3        4   
0                               01           4300.0  5950.0  6775.0  13000.0   
1                               02           3800.0  4800.0  4350.0  20000.0   
2                               03           3600.0  4800.0  5800.0   7925.0   
3                               04           3650.0  5000.0  7200.0  11700.0   
4                               05           3100.0  3800.0  4500.0   6200.0   
5                               06           4500.0  6150.0     NaN      NaN   
6                               07           4250.0  4000.0  5037.5   9500.0   
7                               08           3000.0  3950.0  4700.0   5875.0   
8                               09           3712.5  4750.0  5887.5   8525.0   
9                               10           3450.0  4400.0  6200.0   9500.0   
10                              11           3200.0  4200.0  5400.0   8200.0   
11                              12           2800.0  3500.0  3800.0   5225.0   
12                              13           2900.0  3800.0  4500.0   6000.0   
13                              14           2700.0  3400.0  4000.0   5000.0   
14                              15           2800.0  3600.0  4500.0   6800.0   
15                              16           3000.0  3400.0  4100.0   5800.0   
16                              17           2500.0  3100.0  3525.0   4500.0   
17                              18           2800.0  3300.0  4000.0   5000.0   
18                              19           2700.0  3300.0  4000.0   4650.0   
19                              20           3200.0  4000.0  4500.0   5750.0   
20                              21           2600.0  3300.0  4200.0   5300.0   
21                              22           3300.0  4000.0  4225.0   5500.0   
22                              23           2837.5  3200.0  3712.5   4500.0   
23                              25           2500.0  3000.0  3900.0   2975.0   
24                              26           3000.0  3200.0  3495.0   5400.0   
25                              27           2675.0  3025.0  3725.0   4100.0   
26                              28           2500.0  3200.0  3700.0   4800.0   

                             rental_rent_50th                           \
rental_noOfBedRoom         5                1       2       3        4   
0                   50000.00           4900.0  6850.0  9400.0  16750.0   
1                        NaN           4400.0  5500.0  7500.0  20000.0   
2                   14000.00           3900.0  5300.0  6500.0   9000.0   
3                   10662.50           3900.0  5800.0  8500.0  14500.0   
4                    7670.25           3425.0  4200.0  5500.0   6900.0   
5                        NaN           4963.0  6300.0     NaN      NaN   
6                   30000.00           5000.0  4800.0  5900.0  11000.0   
7                    8500.00           3200.0  4300.0  5500.0   6650.0   
8                   11200.00           4200.0  5700.0  7500.0  11000.0   
9                    9625.00           3800.0  5000.0  7500.0  13400.0   
10                   6350.00           3500.0  4925.0  6300.0  10000.0   
11                   7750.00           3050.0  4000.0  4650.0   6100.0   
12                       NaN           3100.0  4200.0  5200.0   6500.0   
13                  10000.00           3000.0  4100.0  4690.0   5700.0   
14                   4400.00           3200.0  4300.0  5500.0   8000.0   
15                   8625.00           3200.0  3800.0  4700.0   6500.0   
16                       NaN           2900.0  3500.0  4000.0   5000.0   
17                   5825.00           3000.0  3600.0  4500.0   5500.0   
18                   7775.00           3000.0  3650.0  4350.0   5850.0   
19                   7050.00           3700.0  4350.0  5000.0   6700.0   
20                       NaN           2950.0  3900.0  4879.5   6400.0

In [10]:
df.columns = df.columns.swaplevel(0, 1)
df.sort_index(axis=1, level=0, inplace=True)
df

rental_noOfBedRoom                                1                   \
                   rental_district rental_rent_25th rental_rent_50th   
0                               01           4300.0           4900.0   
1                               02           3800.0           4400.0   
2                               03           3600.0           3900.0   
3                               04           3650.0           3900.0   
4                               05           3100.0           3425.0   
5                               06           4500.0           4963.0   
6                               07           4250.0           5000.0   
7                               08           3000.0           3200.0   
8                               09           3712.5           4200.0   
9                               10           3450.0           3800.0   
10                              11           3200.0           3500.0   
11                              12           2800.0           3050.0   
12                              13           2900.0           3100.0   
13                              14           2700.0           3000.0   
14                              15           2800.0           3200.0   
15                              16           3000.0           3200.0   
16                              17           2500.0           2900.0   
17                              18           2800.0           3000.0   
18                              19           2700.0           3000.0   
19                              20           3200.0           3700.0   
20                              21           2600.0           2950.0   
21                              22           3300.0           3500.0   
22                              23           2837.5           3075.0   
23                              25           2500.0           2650.0   
24                              26           3000.0           3000.0   
25                              27           2675.0           3000.0   
26                              28           2500.0           3000.0   

rental_noOfBedRoom                                 2                   \
                   rental_rent_75th rental_rent_25th rental_rent_50th   
0                            5500.0           5950.0           6850.0   
1                            4800.0           4800.0           5500.0   
2                            4200.0           4800.0           5300.0   
3                            4100.0           5000.0           5800.0   
4                            3762.5           3800.0           4200.0   
5                            5200.0           6150.0           6300.0   
6                            5600.0           4000.0           4800.0   
7                            3800.0           3950.0           4300.0   
8                            5000.0           4750.0           5700.0   
9                            4200.0           4400.0           5000.0   
10                           3900.0           4200.0           4925.0   
11                           3350.0           3500.0           4000.0   
12                           3400.0           3800.0           4200.0   
13                           3500.0           3400.0           4100.0   
14                           3600.0           3600.0           4300.0   
15                           3500.0           3400.0           3800.0   
16                           3000.0           3100.0           3500.0   
17                           3200.0           3300.0           3600.0   
18                           3200.0           3300.0           3650.0   
19                           3984.0           4000.0           4350.0   
20                           3275.0           3300.0           3900.0   
21                           3694.0           4000.0           4500.0   
22                           3300.0           3200.0           3600.0   
23                           2800.0           3000.0           3300.0   
24           

In [11]:
df.to_csv('transactions_price_index_01052023.csv', na_rep='N/A', quoting=csv.QUOTE_NONE, index=False)